**# https://learn.microsoft.com/en-us/azure/databricks/getting-started/cleanse-enhance-data**


In [0]:
%sql
use devcatalog.default

In [0]:
%sql
show schemas

In [0]:
catalog = "devcatalog"
schema = "default"
table_name = "baby_names"
silver_table_name = "baby_names_prepared"
gold_table_name = "top_baby_names_2021"
path_table = catalog + "." + schema
print(path_table) # Show the complete path

In [0]:
df_raw=spark.read.table(f"{path_table}.{table_name}")
display(df_raw)

In [0]:
df_raw.count()

In [0]:
from pyspark.sql.functions import col, initcap, when
df_rename_year=df_raw.withColumnRenamed('Year', 'Year_Of_Birth')
df_init_caps=df_rename_year.withColumn('First_Name', initcap(col('First_Name').cast('string')))
df_baby_names_sex=df_init_caps.withColumn('Sex', when(col('Sex')=='M', 'Male').when(col('Sex')=='F', 'Female'))
display(df_baby_names_sex)
df_baby_names_sex.write.mode('overwrite').saveAsTable(f'{path_table}.{silver_table_name}')

In [0]:
from pyspark.sql.functions import expr, sum, desc

df_baby_names_2021_grouped=(
    df_baby_names_sex.where(col('Year_Of_Birth')==2021)
    .groupBy('Sex', 'First_Name')
    .agg(sum('Count').alias('Total_Count'))
    .sort(desc('Total_Count'))
)
display(df_baby_names_2021_grouped)
df_baby_names_2021_grouped.write.mode('overwrite').saveAsTable(f'{path_table}.{gold_table_name}')

In [0]:
display(spark.sql(f'select * from {path_table}.{gold_table_name}'))

Databricks visualization. Run in Databricks to view.